In [1]:
import requests
import urllib
from lxml import etree
import os

In [2]:
# 通过url获取每个图集的链接页面
def getArticleLinks(url):
    html = requests.get(url)
    Selector = etree.HTML(html.text)
    # 通过“Xpath”获取每个帖子的url后缀
    # <div class="image gallery-group-1"><div…><a href…>
    url_list = Selector.xpath('//div[@class="image gallery-group-1"]/div/a/@href')
    # <div class="image gallery-group-1"><div…><a href…><img alt=……>
    # url_list = Selector.xpath('//div[@class="image gallery-group-1"]/div/a/img/@alt')
    # 在每个链接的前加上主链接
    for i in range(len(url_list)):
        url_list[i] = 'http://meizian.com' + url_list[i]
    return url_list

In [3]:
# 需要下载的图集页面数
K = 10

In [4]:
four_impression_all = []
for i in range(0,K):
    four_impression_all[i*10:(i+1)*10-1] = getArticleLinks('http://meizian.com/taotu/heisiai.html?p=%s' %(i+1))

In [5]:
# 通过url获取每个图片的链接
def getPicLinks(url):
    html = requests.get(url)
    Selector = etree.HTML(html.text)
    # 通过“Xpath”获取每个图片的url
    # <div class="image gallery-group-1"><div…><a href…><img src=……>
    url_list = Selector.xpath('//div[@class="image gallery-group-1"]/div/a/img/@src')
    # 在每个链接后加上扩展名“.jpg”，形成图片链接
    for i in range(len(url_list)):
        url_list[i] = url_list[i] + '.jpg'
    # 判断异常数值，再删除
    if 'alt="".jpg' in url_list:
        url_list.remove('alt="".jpg')
    return url_list

In [6]:
four_impression_all[2]

'http://meizian.com/taotu/heisiai/24276.html'

In [7]:
four_impression_all[2][33:].replace(".html", '')

'24276'

In [8]:
page = len(four_impression_all)
page

100

In [9]:
# 为每个图集创建独立文件夹，并下载图片
def download_img(url_list,page):
    # 该目录下创建一个downloads文件夹存放下载图片
    if not os.path.exists('downloads'):
        os.mkdir('downloads')
    root_path = os.getcwd()
    x =1
    for i in range(page):
        img_dir = 'downloads/' + url_list[i-1][32:].replace(".html", '')
        if not os.path.exists(img_dir):
            os.mkdir(img_dir)
        os.chdir(img_dir)
        Pic_Links = getPicLinks(url_list[i-1])
        for img in Pic_Links:
            urllib.urlretrieve(img, 'heisiai_%s.jpg' %x)
            x += 1
        os.chdir(root_path)

In [10]:
download_img(four_impression_all,page)

In [11]:
# 用于补充一些掉线、断点、没下载完的图库
meizian = getPicLinks('http://meizian.com/taotu/heisiai/24204.html')
x = 2421
for img in meizian:
    urllib.urlretrieve(img, 'downloads/24204/heisiai_%s.jpg' %x)
    x += 1